In [2]:

from sklearn.model_selection import train_test_split

from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold

import numpy as np
from moabb.paradigms import MotorImagery
from sklearn import preprocessing
from braindecode.datasets import MOABBDataset


# import adabound

import torch
import torch.nn as nn 
from torch import tensor



In [3]:
from utils import utils
from utils import data_utils

In [4]:
utils.set_seed(0)
device = utils.set_device()

In [5]:
from moabb import datasets

In [6]:
# data config
dataset_name="BNCI2015001"
subject_id=1

paradigm = MotorImagery(n_classes=2)
dataset = getattr(datasets, "BNCI2015001")
subjects = [subject_id]

# load data
X, y, metadata = paradigm.get_data(dataset=dataset(), subjects=subjects)
X = X * 1000
le = preprocessing.LabelEncoder()
le.fit(y)
y = le.transform(y)

# load metadata
ds = MOABBDataset(dataset_name=dataset_name, subject_ids=subject_id)
ch_names = ds.datasets[0].raw.info["ch_names"]
samplerate = ds.datasets[0].raw.info["sfreq"]



Choosing from all possible events


200 events found
Event IDs: [1 2]
200 events found
Event IDs: [1 2]


In [7]:
splitter = getattr(data_utils, "partition_"+dataset_name)
X_, X_test, y_, y_test = splitter(X, y, metadata)

In [8]:
import wandb

wandb.login()

wandb: Currently logged in as: mirfarid (use `wandb login --relogin` to force relogin)


True

In [9]:
from models.model import *
from models.loss import csp_loss

In [10]:
import optuna
from optuna.integration.wandb import WeightsAndBiasesCallback

In [11]:
from utils.model_utils import get_config, get_optim

In [12]:
trial_num = -1 # trial number of optuna

def objective(trial):
    global trial_num
    trial_num += 1
    #hyperparameters
    n_components = 2
    component_order = 'alternate'
    
    # get parameters specific to the current trial
    
    config = get_config(trial=trial)
        
    # definde model and classifier
    clf = Classifier(n_components, config['filters']).to(device)
    model = Tception(samplerate, config['filters']).to(device)
    CSPloss = csp_loss(n_components = n_components,
                       n_channels = 13,
                       device=device,
                       component_order = component_order)

    # get optimizers specific to the current trial
    
    optimizer_model, optimizer_clf = get_optim(config, model, clf)
    criterion = nn.BCELoss()    

    params = {'n_components': n_components,
          'filters': config['filters'],
          'component_order': component_order,
          'optim_type_model': config['optimizer_name_model'],
          'weight_decay_model': config['weight_decay_model'],
          'lr_model': config['lr_model'],
          'optim_type_clf': config['optimizer_name_clf'],
          'weight_decay_clf': config['weight_decay_clf'],
          'lr_clf': config['lr_clf'],
          'trial_id': trial_num}
    wandb.init(name='trial'+str(trial_num),
               config=params,
               project=dataset_name)   

    val_losses, accs = [], []   
       
    X_, X_test, y_, y_test = splitter(X, y, metadata)
    skf = StratifiedKFold(n_splits=5)
    for run_ind, (train_index, val_ind) in enumerate(skf.split(X_, y_)):

        X_train, X_valid = X_[train_index], X_[val_ind]
        y_train, y_valid = y_[train_index], y_[val_ind]

        X_train = torch.as_tensor(X_train).float().unsqueeze(1).to(device)
        y_train = torch.as_tensor(y_train).float().to(device)
        X_val = torch.as_tensor(X_valid).float().unsqueeze(1).to(device)
        y_val = torch.as_tensor(y_valid).float().to(device)
        
        clf.reset_parameters()
        model.reset_parameters()

        for epoch in tqdm(range(5)):

            
            model.train()
            clf.train()

            optimizer_model.zero_grad()

            H = model(X_train)
            cpsloss_train , transformed = CSPloss(H, y_train)

            cpsloss_train.backward(retain_graph=True)
            optimizer_clf.zero_grad()

            outputs = clf(transformed.clone().detach())
            loss_train = criterion(outputs, y_train)


            loss_train.backward()
            optimizer_model.step()
            optimizer_clf.step()

            accuracy_train = ((outputs > 0.5) == y_train).float().mean()
            model.eval()
            clf.eval()

            with torch.no_grad():
                H = model(X_val)

                cpsloss_val, transformed = CSPloss.filter_bank_val(H, y_val)

                outputs = clf(transformed.clone().detach())
                loss_val = criterion(outputs, y_val)
                accuracy_val = ((outputs > 0.5) == y_val).float().mean()


            val_losses.append(loss_val)
            accs.append(accuracy_val)
            wandb.log({
                'train_accuracy': accuracy_train,
                'train_loss': loss_train,
                'train_csploss': cpsloss_train,
                'valid_accuracy': accuracy_val,
                'valid_loss': loss_val,
                'valid_csploss': cpsloss_val,
                'run': run_ind,
                'epoch': epoch
            })
        

    loss, acc = tensor(val_losses), tensor(accs)
    loss, acc = loss.view(5, 5), acc.view(5, 5)
    loss, argmin = loss.min(dim=1)
    acc = acc[torch.arange(5, dtype=torch.long), argmin]
    loss_mean = loss.mean().item()
    acc_mean = acc.mean().item()
    acc_std = acc.std().item()
    wandb.log({
        'cv_acc_mean': acc_mean,
        'cv_acc_std': acc_std,
        'cv_loss_mean': loss_mean,
        'trial': trial_num
        })

    # Handle pruning based on the intermediate value.
    if trial.should_prune():
        raise optuna.exceptions.TrialPruned()


    return acc_mean

In [14]:
# wandb_kwargs = {"project": "bci-second-test"}
# wandbc = WeightsAndBiasesCallback(metric_name="5foldValidAccuracy", wandb_kwargs=wandb_kwargs)

study = optuna.create_study(direction="maximize", study_name="my_study")
study.optimize(objective, n_trials=10)

[I 2022-01-04 03:20:13,212] A new study created in memory with name: my_study


  0%|          | 0/5 [00:00<?, ?it/s]/home/puppy/projects/BCI/models/loss.py:89: UserWarning: torch.cholesky is deprecated in favor of torch.linalg.cholesky and will be removed in a future PyTorch release.
L = torch.cholesky(A)
should be replaced with
L = torch.linalg.cholesky(A)
and
U = torch.cholesky(A, upper=True)
should be replaced with
U = torch.linalg.cholesky(A).transpose(-2, -1).conj().
This transform will produce equivalent results for all valid (symmetric positive definite) inputs. (Triggered internally at  /opt/conda/conda-bld/pytorch_1639180487213/work/aten/src/ATen/native/BatchLinearAlgebra.cpp:1285.)
  L = torch.cholesky(S_t, upper=False)
/home/puppy/projects/BCI/models/loss.py:90: UserWarning: torch.solve is deprecated in favor of torch.linalg.solveand will be removed in a future PyTorch release.
torch.linalg.solve has its arguments reversed and does not return the LU factorization.
To get the LU factorization see torch.lu, which can be used with torch.lu_solve or torch.